In [118]:
import pandas as pd
import numpy as np
import sys
import random
from sklearn.linear_model import LinearRegression
import sklearn as sk
from skopt import gp_minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.gaussian_process.kernels import Matern
import time

See ./2_pipeline/3_simulation1D2_trueValues for the generative model

In [311]:
obsData = generateObservedData(100, 1, [-0.5, 1])
obsData = pd.DataFrame(obsData).transpose()
obsData = obsData.rename(columns = {0:"intercept", 1:"X", 2:"A", 3:"Y"})
print(computeIPW([0.5, .75], obsData, 100))
print(computeStabilizedIPW([0.5, 0.75], obsData, 100))
print(computeRegEst([0.5, 0.75], obsData, 100))
print(computeAIPWE([0.5, 0.75], obsData, 100))

-0.13498793065284917
-0.16873491331606147
0.1063004123981152
-0.07419271605872826


In [285]:
# Generate observed data
def generateObservedData(sampleSize, w, gamma):
    intercept = np.ones(sampleSize)
    X = w*np.random.uniform(0, 1, sampleSize)
    A = random.choices([0, 1], weights = [0.5, 0.5], k = sampleSize)
    Y = gamma[0]*intercept + gamma[1]*X + A*np.cos(X*2*np.pi)
    return intercept, X, A, Y
    
# IPW estimate (negated)
def computeIPW(beta, obsData, n):
    obsData['A_d'] = np.logical_or(obsData['X'] < beta[0], obsData['X'] > beta[1])*1.0
    obsData['C_d'] = np.where(obsData['A'] == obsData['A_d'], 1, 0)
    obsData['pi_d'] = 0.5
    obsData['summand'] = obsData['C_d']*obsData['Y']/obsData['pi_d']
    
    # Estimate the value
    vhat_ipw = sum(obsData['summand'])/(n)
    
    return -1*vhat_ipw

# Stabilized IPW estimate (negated)
def computeStabilizedIPW(beta, obsData, n):
    obsData['A_d'] = np.logical_or(obsData['X'] < beta[0], obsData['X'] > beta[1])*1.0
    obsData['C_d'] = np.where(obsData['A'] == obsData['A_d'], 1, 0)
    obsData['pi_d'] = 0.5
    obsData['summand'] = obsData['C_d']*obsData['Y']*obsData['pi_d']
    
    # Estimate the value
    vhat_stabilizedIPW = sum(obsData['summand'])/(sum(obsData['C_d']*obsData['pi_d']))
    
    return -1*vhat_stabilizedIPW

# Regression estimator (G-computation) (negated)
def computeRegEst(beta, obsData, n):
    # X contains the covariates
    X = obsData.loc[:,['X', 'A']]
    X['int'] = np.ones(n)
    X['AX'] = np.multiply(obsData['X'], obsData['A'])
    X = X.loc[:, ['int', 'X', 'A', 'AX']]
    
    # Y contains the outcomes
    Y = obsData['Y']
    
    # Fit the regression model
    model = LinearRegression().fit(X, Y)
    
    # Calculate Qhat(H, 1)
    X_1 = X.copy(deep = True)
    X_1['A'] = 1
    Qhat1 = model.predict(X_1)
    
    # Calculate Qhat(H, 0)
    X_0 = X.copy(deep = True)
    X_0['A'] = 0
    Qhat0 = model.predict(X_0)
    
    # Calculate the treatment recommendation under the policy indexed by beta
    A_d = np.logical_or(obsData['X'] < beta[0], obsData['X'] > beta[1])*1.0
    
    # Estimate the value
    vhat = np.sum(np.where(A_d == 1, Qhat1, 0) + np.where(A_d == 0, Qhat0, 0))*(1/n)
    
    return -1*vhat

# AIPWE (negated)
def computeAIPWE(beta, obsData, n):
    # IPW piece
    obsData['A_d'] = np.logical_or(obsData['X'] < beta[0], obsData['X'] > beta[1])*1.0
    obsData['C_d'] = np.where(obsData['A'] == obsData['A_d'], 1, 0)
    obsData['pi_d'] = 0.5
#     obsData['summand'] = obsData['C_d']*obsData['Y']*obsData['pi_d']
#     obsData['weight'] = (obsData['C_d'] - obsData['pi_d'])/obsData['pi_d']
    
    # Regression piece
    # X contains the covariates
    X = obsData.loc[:,['X', 'A']]
    X['int'] = np.ones(n)
    X['AX'] = np.multiply(obsData['X'], obsData['A'])
    X = X.loc[:, ['int', 'X', 'A', 'AX']]
    
    # Y contains the outcomes
    Y = obsData['Y']
    
    # Fit the regression model
    model = LinearRegression().fit(X, Y)
    
    # Calculate Qhat(H, 1)
    X_1 = X.copy(deep = True)
    X_1['A'] = 1
    Qhat1 = model.predict(X_1)
    
    # Calculate Qhat(H, 0)
    X_0 = X.copy(deep = True)
    X_0['A'] = 0
    Qhat0 = model.predict(X_0)
    
    # Calculate the treatment recommendation under the policy indexed by beta
    A_d = np.logical_or(obsData['X'] < beta[0], obsData['X'] > beta[1])*1.0
    
    # Calculate the pseudo value
    obsData['yhat'] = np.where(A_d == 1, Qhat1, 0) + np.where(A_d == 0, Qhat0, 0)
    
    # Estimate the value
#     vhat = (1/n)*sum(obsData['summand'] - obsData['weight']*obsData['yhat'])
    vhat = (1/n)*sum(obsData['yhat']) + (1/n)*sum((obsData['C_d']/obsData['pi_d'])*(obsData['Y'] - obsData['yhat']))
#     vhat = (1/n)*sum((obsData['C_d']*(1/obsData['pi_d'])*obsData['Y']) - (1 - obsData['C_d']*(1/obsData['pi_d']))*obsData['yhat'])
#     vhat = (1/n)*sum(obsData['Y']*obsData['C_d']*(1/obsData['pi_d']) - ((obsData['C_d'] - obsData['pi_d'])/obsData['pi_d'])*obsData['yhat'])
    
    return -1*vhat
    

Simulation scenarios - Simulation 1, Part 1
-------------------------------- 
Value estimation method | n | CV | nSims
---|---|---|---|
IPW| 200| 0.75| 1000|
IPW | 500 | 0.75 | 1000
IPW | 1000 | 0.75 | 1000
IPW | 200 | 1.0 | 1000
IPW | 500 | 1.0 | 1000
IPW | 1000 | 1.0 | 1000
IPW | 200 | 1.25 | 1000
IPW | 500 | 1.25 | 1000
IPW | 1000 | 1.25 | 1000
Stabilized IPW| 200| 0.75| 1000
Stabilized IPW | 500 | 0.75 | 1000
Stabilized IPW | 1000 | 0.75 | 1000
Stabilized IPW | 200 | 1.0 | 1000
Stabilized IPW | 500 | 1.0 | 1000
Stabilized IPW | 1000 | 1.0 | 1000
Stabilized IPW | 200 | 1.25 | 1000
Stabilized IPW | 500 | 1.25 | 1000
Stabilized IPW | 1000 | 1.25 | 1000
G-computation| 200| 0.75| 1000
G-computation | 500 | 0.75 | 1000
G-computation | 1000 | 0.75 | 1000
G-computation | 200 | 1.0 | 1000
G-computation | 500 | 1.0 | 1000
G-computation | 1000 | 1.0 | 1000
G-computation | 200 | 1.25 | 1000
G-computation | 500 | 1.25 | 1000
G-computation | 1000 | 1.25 | 1000
AIPWE | 200| 0.75| 1000
AIPWE | 500 | 0.75 | 1000
AIPWE | 1000 | 0.75 | 1000
AIPWE | 200 | 1.0 | 1000
AIPWE | 500 | 1.0 | 1000
AIPWE | 1000 | 1.0 | 1000
AIPWE | 200 | 1.25 | 1000
AIPWE | 500 | 1.25 | 1000
AIPWE | 1000 | 1.25 | 1000

Simulation steps
-----------------------
1. Set the seed
2. Generate a simulation data set. Call it `ObsData`.
3. Fit GP and optimize.
    * Collect the parameters for the estimated optimal DTR
    * Collect the estimated value for the estimated optimal DTR
    * Collect the evaluation points for the GP in (4)
4. Fit a GP to the evaluation points
    * Compute the predicted values over a fixed grid
    * Compute the distance between the truth and predicted V
        * sup norm
        * l1, l2
        * Set error (estimated set and true set): Percent of overlap. Intersection of the two plus the intersection of the complement. Divide by the whole area. If the sum of the two is 100% then you have perfect alignment. (like integrated hamming distance)
5. Return the following
    * Parameters for the estimated optimal DTR
    * Estimate for the value of the optimal DTR
    * Distance between predicted values and the truth across the parameter space (metric induced by the sup norm, l1 norm, and l2 norm)

"Wrapped" version

In [125]:
# Simulation code --------------------------------------------------------------
w = 1 #[0.75, 1, 1.25][int(sys.argv[3])]
# Read in the true values for the evaluation of the GP
trueValues = pd.read_csv('../2_pipeline/3_simulation1D3_'+str(w)+'_trueValues.csv')
nObs = 500 #int(sys.argv[2])
evaluationEstimator = "AIPWE"#sys.argv[1]
L = 10#1000
outFileName = '3_simulation1D3_'+str(evaluationEstimator)+'_'+str(nObs)+'_'+str(w)+'.csv'

# Places to hold the things we want to keep
optDTR_param_holder = []
optDTR_value_holder = []
L_holder = []
norm_sup_holder = []
norm_1_holder = []
norm_2_holder = []

# Set the seed -------------------------------------
np.random.seed(1234)

for l in range(L):
    
    # Generate simulation data set ---------------------
    # Parameters for data generation
    gamma = [-0.5, 1]
    obsData = generateObservedData(nObs, w, gamma)
    obsData = pd.DataFrame(obsData).transpose()
 
    # Tidy up the dataframe with the "observed data"
    obsData = obsData.rename(columns = {0:'intercept', 1:'X', 2:'A', 3:'Y'})

    # Bayesian optimization ------------------------------
    noise = 0.01
    if evaluationEstimator == 'IPW':
        def computeIPW_internal(beta, obsData = obsData, nObs = nObs):
            return computeIPW(beta, obsData = obsData, n = nObs)
        ei_result = gp_minimize(computeIPW_internal,
                   [(0.0, 1.0), (0.0, 1.0)],
                   acq_func = "EI",
                   n_calls = 25,
                   n_random_starts = 25,
                   noise = noise)
    if evaluationEstimator == "sIPW":
        def computeStabilizedIPW_internal(beta, obsData = obsData, nObs = nObs):
            return computeStabilizedIPW(beta, obsData = obsData, n = nObs)
        ei_result = gp_minimize(computeStabilizedIPW_internal,
                   [(0.0, 1.0), (.0, 1.0)],
                   acq_func = "EI",
                   n_calls = 25,
                   n_random_starts = 25,
                   noise = noise)
    if evaluationEstimator == "gcomp":
        def computeRegEst_internal(beta, obsData = obsData, nObs = nObs):
            return computeRegEst(beta, obsData = obsData, n = nObs)
        ei_result = gp_minimize(computeRegEst_internal,
                   [(0.0, 1.0), (0.0, 1.0)],
                   acq_func = "EI",
                   n_calls = 25,
                   n_random_starts = 25,
                   noise = noise)
    if evaluationEstimator == "AIPWE":
        def computeAIPWE_internal(beta, obsData = obsData, nObs = nObs):
            return computeAIPWE(beta, obsData = obsData, n = nObs)
        ei_result = gp_minimize(computeAIPWE_internal,
                   [(0.0, 1.0), (0.0, 1.0)],
                   acq_func = "EI",
                   n_calls = 25,
                   n_random_starts = 25,
                   noise = noise)
    # Extract the relevant information
    optDTR_param = ei_result['x']
    optDTR_value = ei_result['fun']
    evaluation_X = ei_result['x_iters']
    evaluation_Y = ei_result['func_vals']

    # Fit a GP to the evaluation points --------------------
    kernel = 1.0 * Matern(length_scale = [1.0, 1.0], nu = 1.0) \
        + WhiteKernel(noise_level = 10, noise_level_bounds = (1e-10, 1e2))
    gpr = GaussianProcessRegressor(kernel = kernel, alpha = 0.0)
    gpr.fit(evaluation_X, evaluation_Y)

    # Get predictions across a fine grid of the parameter space
    diffs = []
    for i in range(trueValues.shape[0]):
        pred = gpr.predict(np.array(trueValues.loc[i, ['beta0', 'beta1']]).reshape(1, -1))
        diffs.append(-1*pred - trueValues.loc[i, 'value'])
    # Compute the distance between the prediction and the Truth (actual truth, not the evaluation truth)    
    norm_sup = max(np.abs(diffs))
    norm_1 = np.mean(np.abs(diffs))
    norm_2 = np.sqrt(sum(np.abs(diffs)**2)*(1/trueValues.shape[0]))

    # Update the holder lists
    optDTR_param_holder.append(optDTR_param)
    optDTR_value_holder.append(optDTR_value)
    L_holder.append(l)
    norm_sup_holder.append(norm_sup)
    norm_1_holder.append(norm_1)
    norm_2_holder.append(norm_2)

# Convert the list of arrays to a list
norm_sup_holder = [i[0] for i in norm_sup_holder]
norm_2_holder = [i[0] for i in norm_2_holder]

# Put the lists into a data frame
out1 = pd.DataFrame({'l':L_holder, 'optDTR_value':optDTR_value_holder, 'norm_sup':norm_sup_holder,
                     'norm_1':norm_1_holder, 'norm_2':norm_2_holder})
out2 = pd.DataFrame(optDTR_param_holder, columns = ['beta0', 'beta1'])
out = pd.concat([out1, out2], axis = 1)

# Write the dataframe to a csv
#out.to_csv('../2_pipeline/'+outFileName)    

/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-10. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-10. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


In [126]:
out

,l,optDTR_value,norm_sup,norm_1,norm_2,beta0,beta1
0,0,-0.536863,0.718937,0.457065,0.480345,0.793085,0.151083
1,1,-0.530240,0.694729,0.461934,0.481883,0.819632,0.524242
2,2,-0.543167,0.723258,0.467631,0.489312,0.760961,0.425949
3,3,-0.533890,0.698902,0.435803,0.465234,0.971643,0.772966
4,4,-0.540789,0.701231,0.455621,0.479360,0.949769,0.547466
5,5,-0.544703,0.731106,0.480157,0.502452,0.717437,0.940285
6,6,-0.538743,0.730007,0.460133,0.482958,0.265613,0.740061
7,7,-0.536895,0.702270,0.456437,0.479683,0.535691,0.917510
8,8,-0.535390,0.700650,0.438820,0.467161,0.375056,0.799146
9,9,-0.522980,0.751412,0.451796,0.473286,0.515916,0.217259


In [ ]:
generateObservedData(10, 1.11, [0.5, 1])